<a href="https://colab.research.google.com/github/cs7641-team4/cs7641project/blob/main/Lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis

In [31]:
#import numpy as np

import warnings
warnings.filterwarnings(action='once')


import pyLDAvis

import gensim
from gensim import corpora
from gensim.models import CoherenceModel

import pyLDAvis.gensim_models as gensimvis

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
nltk.download('wordnet')

import pandas as pd



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#get url of dataset and load it into a dataframe
url = 'https://raw.githubusercontent.com/cs7641-team4/cs7641project/main/data/EMNLP2020.csv'
data = pd.read_csv(url)
data.head()

,title,abstract,authors,pdf_url,url,label1,label2
0,Blank Language Models,"We propose Blank Language Model (BLM), a model...","['Tianxiao Shen', 'Victor Quach', 'Regina Barz...",https://www.aclweb.org/anthology/2020.emnlp-ma...,https://www.aclweb.org/anthology/2020.emnlp-ma...,2.0,4.0
1,Acrostic Poem Generation,We propose a new task in the area of computati...,"['Rajat Agarwal', 'Katharina Kann']",https://www.aclweb.org/anthology/2020.emnlp-ma...,https://www.aclweb.org/anthology/2020.emnlp-ma...,2.0,NaN
2,Unsupervised Adaptation of Question Answering ...,BERT-era question answering systems have recen...,"['Steven Rennie', 'Etienne Marcheret', 'Neil M...",https://www.aclweb.org/anthology/2020.emnlp-ma...,https://www.aclweb.org/anthology/2020.emnlp-ma...,5.0,NaN
3,An Imitation Game for Learning Semantic Parser...,"Despite the widely successful applications, bo...","['Ziyu Yao', 'Yiqi Tang', 'Wen-tau Yih', 'Huan...",https://www.aclweb.org/anthology/2020.emnlp-ma...,https://www.aclweb.org/anthology/2020.emnlp-ma...,8.0,1.0
4,Conversational Semantic Parsing,The structured representation for semantic par...,"['Armen Aghajanyan', 'Jean Maillard', 'Akshat ...",https://www.aclweb.org/anthology/2020.emnlp-ma...,https://www.aclweb.org/anthology/2020.emnlp-ma...,8.0,1.0


In [4]:
#create list of documents where each document is an abstract

doc_list = data['abstract'].to_list()
doc_list[:3]

['We propose Blank Language Model (BLM), a model that generates sequences by dynamically creating and filling in blanks. The blanks control which part of the sequence to expand, making BLM ideal for a variety of text editing and rewriting tasks. The model can start from a single blank or partially completed text with blanks at specified locations. It iteratively determines which word to place in a blank and whether to insert new blanks, and stops generating when no blanks are left to fill. BLM can be efficiently trained using a lower bound of the marginal data likelihood. On the task of filling missing text snippets, BLM significantly outperforms all other baselines in terms of both accuracy and fluency. Experiments on style transfer and damaged ancient text restoration demonstrate the potential of this framework for a wide range of applications.',
 'We propose a new task in the area of computational creativity: acrostic poem generation in English. Acrostic poems are poems that contain

In [5]:
#create function to parse the text

def parse_text(txt):
  txt = txt.lower()
  txt = re.sub(r'[^a-z ]', ' ', txt)
  tokenized_words=word_tokenize(txt)

  filtered_words = [w for w in tokenized_words if not w in stop_words]

  lemmitized_words = [lemmatizer.lemmatize(w) for w in filtered_words]

  tagged = nltk.pos_tag(lemmitized_words) 
  nouns = [tup[0] for tup in tagged if tup[1][:2] == 'NN']
  # verbs = [tup[0] for tup in tagged if tup[1][:2] == 'VB']
  # adj_adv = [tup[0] for tup in tagged if tup[1][:2] == 'VB' or tup[1][:2] == 'JJ']
  # adj_noun = [tup[0] for tup in tagged if tup[1][:2] == 'NN' or tup[1][:2] == 'JJ']

  return nouns



In [6]:
#set stopwords to english 
stop_words = set(stopwords.words('english')) 

#clean the documents based on parse function
doc_clean = list(map(lambda x: parse_text(x), doc_list))

doc_clean[:1]

[['language',
  'model',
  'blm',
  'model',
  'blank',
  'part',
  'sequence',
  'expand',
  'variety',
  'text',
  'task',
  'start',
  'blank',
  'blank',
  'location',
  'word',
  'place',
  'stop',
  'fill',
  'data',
  'likelihood',
  'task',
  'text',
  'snippet',
  'blm',
  'term',
  'accuracy',
  'fluency',
  'experiment',
  'style',
  'transfer',
  'text',
  'restoration',
  'framework',
  'range',
  'application']]

In [7]:
#construct dictionary mapping unique words to an index
dictionary = corpora.Dictionary(doc_clean)
#create matrix where every row is a document and the columns are each unique term (BOW format)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in doc_clean]

In [13]:
doc_term_matrix[1] #the first number in the tuple is the index identifier for the word and the second is the number of times that word occurs

[(4, 1),
 (6, 1),
 (10, 1),
 (13, 5),
 (23, 5),
 (28, 3),
 (29, 1),
 (30, 2),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 2),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 3),
 (40, 1),
 (41, 2),
 (42, 2),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 8),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 2),
 (52, 2),
 (53, 1)]

In [15]:
#the first number in the tuple is the index identifier for the word and the second is the number of times that word occurs
#for example word index 46 occurs the most in the second abstract
dictionary[46]
#view the first abstract
#dictionary[ind] for (ind, freq) in doc_term_matrix[1]]
doc_clean[1]

['task',
 'area',
 'creativity',
 'poem',
 'generation',
 'poem',
 'poem',
 'contain',
 'message',
 'letter',
 'line',
 'spell',
 'word',
 'phrase',
 'define',
 'task',
 'generation',
 'task',
 'constraint',
 'input',
 'word',
 'letter',
 'line',
 'spell',
 'word',
 'poem',
 'semantics',
 'poem',
 'baseline',
 'model',
 'task',
 'language',
 'model',
 'combination',
 'model',
 'datasets',
 'generation',
 'training',
 'data',
 'task',
 'topic',
 'prediction',
 'model',
 'topic',
 'poem',
 'experiment',
 'show',
 'poem',
 'baseline',
 'quality',
 'constraint',
 'confirm',
 'poem',
 'model',
 'boost',
 'performance']

In [18]:
#number of documents
n = len(doc_clean)
n

1450

In [41]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
# lda_model = LDA(corpus=doc_term_matrix,
#                 id2word=dictionary,
#                 num_topics=3,
#                 random_state=1,
#                 update_every=10,
#                 alpha='auto',
#                 chunksize=int(n/3),
#                 per_word_topics=True,
#                 passes=30)


lda_model = LDA(corpus=doc_term_matrix,
                id2word=dictionary,
                num_topics=11,
                random_state=1,
                update_every=10,
                alpha='auto',
                chunksize=int(n/3),
                per_word_topics=True,
                passes=30)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [42]:
lda_model.print_topics()

[(0,
  '0.045*"model" + 0.023*"language" + 0.020*"task" + 0.020*"data" + 0.014*"sentence" + 0.012*"method" + 0.012*"performance" + 0.011*"state" + 0.010*"datasets" + 0.010*"translation"'),
 (1,
  '0.048*"model" + 0.026*"language" + 0.024*"task" + 0.016*"network" + 0.014*"knowledge" + 0.013*"information" + 0.013*"approach" + 0.012*"method" + 0.011*"performance" + 0.010*"show"'),
 (2,
  '0.053*"game" + 0.012*"system" + 0.010*"data" + 0.010*"equilibrium" + 0.009*"cache" + 0.008*"engine" + 0.008*"model" + 0.008*"problem" + 0.008*"storage" + 0.007*"device"'),
 (3,
  '0.027*"system" + 0.020*"model" + 0.012*"paper" + 0.010*"time" + 0.009*"approach" + 0.008*"video" + 0.007*"image" + 0.007*"method" + 0.007*"problem" + 0.007*"algorithm"'),
 (4,
  '0.016*"level" + 0.011*"interaction" + 0.011*"system" + 0.011*"customer" + 0.010*"robot" + 0.008*"show" + 0.007*"vehicle" + 0.007*"service" + 0.007*"paper" + 0.007*"skill"'),
 (5,
  '0.022*"problem" + 0.015*"system" + 0.013*"time" + 0.011*"algorithm" + 

In [46]:
#import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
viz = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.197909  0.006879       1        1  24.759680
1     -0.183825  0.009255       2        1  20.735191
5      0.082144 -0.059629       3        1  11.817808
9      0.034895 -0.044864       4        1   8.460808
3      0.012964 -0.062326       5        1   7.943849
8     -0.025013 -0.080682       6        1   7.371619
10    -0.100846  0.047791       7        1   5.371090
7      0.063052  0.183977       8        1   4.596480
6      0.094423 -0.066425       9        1   4.130299
2      0.105557 -0.027123      10        1   3.166557
4      0.114558  0.093147      11        1   1.646619, topic_info=         Term        Freq       Total Category  logprob  loglift
23       task  968.000000  968.000000  Default  30.0000  30.0000
216      game  144.000000  144.000000  Default  29.0000  29.0000
4        data  879.000000  879.000000  Default  28.0000  28.0000
83     system  725.000000  725.000000  Default  27.0000  27.0000
10   language  912.000000  912.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
198   network    6.363853  480.072578  Topic11  -5.2925  -0.2169
189      time    5.993156  437.676803  Topic11  -5.3525  -0.1844
136     point    5.333771  170.157728  Topic11  -5.4691   0.6438
28       word    5.306895  336.543155  Topic11  -5.4741  -0.0433
10   language    5.215490  912.892684  Topic11  -5.4915  -1.0585

[830 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
5469      8  0.916927            ab
1402      1  0.901897          absa
4506      3  0.158738  acceleration
4506      6  0.714320  acceleration
4506      7  0.079369  acceleration
...     ...       ...           ...
433       8  0.323182             x
433       9  0.124301             x
433      10  0.062150             x
2210     11  0.683663           xel
2277      7  0.875102         xglue

[2216 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 6, 10, 4, 9, 11, 8, 7, 3, 5])

In [47]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)



Perplexity:  -7.162368263755333

Coherence Score:  0.3349221464707734


In [54]:
import numpy as np

### SVM on LDA output
The probability distribution for each abstract over the 11 topics can be used as a feature representation of the abstract

Try SVM on this engineered feature to see its performance

In [48]:
doc2topic = lda_model.get_document_topics(doc_term_matrix, minimum_probability=0)
doc2topic[0]

[(0, 0.44728807),
 (1, 0.0011041806),
 (2, 0.000253319),
 (3, 0.00045880914),
 (4, 0.2971512),
 (5, 0.0019241971),
 (6, 0.0012755963),
 (7, 0.00033436264),
 (8, 0.0004657108),
 (9, 0.00051435427),
 (10, 0.24923019)]

In [52]:
#get doc2topic into correct form for using sklearn
docTopicList = [[col[1] for col in row] for row in doc2topic]
docTopicList[0]

[0.44733825,
 0.0011041806,
 0.000253319,
 0.00045880914,
 0.2971482,
 0.0019241971,
 0.0012755963,
 0.00033436264,
 0.0004657108,
 0.00051435427,
 0.24918304]

In [59]:
docTopicArr = np.array(docTopicList)
#get labels
labels = data['label1'].values
docTopicArr.shape, labels.shape

((1450, 11), (1450,))

In [76]:
#store in pandas dataframe to clean data if needed
docTopicDf = pd.DataFrame(np.hstack((docTopicArr, labels.reshape(-1,1))))
docTopicDf.dropna(inplace=True)
docTopicDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1327 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1327 non-null   float64
 1   1       1327 non-null   float64
 2   2       1327 non-null   float64
 3   3       1327 non-null   float64
 4   4       1327 non-null   float64
 5   5       1327 non-null   float64
 6   6       1327 non-null   float64
 7   7       1327 non-null   float64
 8   8       1327 non-null   float64
 9   9       1327 non-null   float64
 10  10      1327 non-null   float64
 11  11      1327 non-null   float64
dtypes: float64(12)
memory usage: 134.8 KB


In [ ]:
#there are some null classes 

In [65]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [78]:
#split data into test and train
X_train, X_test, y_train, y_test = train_test_split(docTopicDf.iloc[:, :11].values, docTopicDf.iloc[:, 11].values, test_size=0.2, random_state=1)

In [94]:
#Create a svm Classifier
clf = svm.SVC(kernel='rbf') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#predict response for training set
y_pred_train = clf.predict(X_train)


# Model Accuracy: how often is the classifier correct?
print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

#classification report
print(metrics.classification_report(y_train, y_pred_train))

# confusion matrix
metrics.confusion_matrix(y_train, y_pred_train)

Train Accuracy: 0.6098020735155514
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        50
         2.0       0.28      0.63      0.39       110
         3.0       0.00      0.00      0.00        22
         4.0       0.26      0.41      0.31        93
         5.0       0.00      0.00      0.00        46
         6.0       0.00      0.00      0.00        40
         7.0       0.00      0.00      0.00        21
         8.0       0.00      0.00      0.00        48
         9.0       0.00      0.00      0.00         9
        10.0       0.00      0.00      0.00        57
        11.0       0.81      0.96      0.88       565

    accuracy                           0.61      1061
   macro avg       0.12      0.18      0.14      1061
weighted avg       0.48      0.61      0.53      1061



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  0,  25,   0,  10,   0,   0,   0,   0,   0,   0,  15],
       [  0,  69,   0,  22,   0,   0,   0,   0,   0,   0,  19],
       [  0,  10,   0,   4,   0,   0,   0,   0,   0,   0,   8],
       [  0,  33,   0,  38,   0,   0,   0,   0,   0,   0,  22],
       [  0,  21,   0,  11,   0,   0,   0,   0,   0,   0,  14],
       [  0,  15,   0,   9,   0,   0,   0,   0,   0,   0,  16],
       [  0,   5,   0,  11,   0,   0,   0,   0,   0,   0,   5],
       [  0,  22,   0,  17,   0,   0,   0,   0,   0,   0,   9],
       [  0,   2,   0,   3,   0,   0,   0,   0,   0,   0,   4],
       [  0,  31,   0,  11,   0,   0,   0,   0,   0,   0,  15],
       [  0,  12,   0,  13,   0,   0,   0,   0,   0,   0, 540]])

In [95]:
#Predict the response for test dataset
y_pred_test = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred_test))

#classification report
print(metrics.classification_report(y_test, y_pred_test))

# confusion matrix
metrics.confusion_matrix(y_test, y_pred_test)

Test Accuracy: 0.5789473684210527
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        20
         2.0       0.33      0.54      0.41        39
         3.0       0.00      0.00      0.00         8
         4.0       0.17      0.26      0.20        23
         5.0       0.00      0.00      0.00        10
         6.0       0.00      0.00      0.00         9
         7.0       0.00      0.00      0.00         2
         8.0       0.00      0.00      0.00        11
        10.0       0.00      0.00      0.00         9
        11.0       0.77      0.94      0.84       135

    accuracy                           0.58       266
   macro avg       0.13      0.17      0.15       266
weighted avg       0.45      0.58      0.51       266



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  0,   7,   0,   4,   0,   0,   0,   0,   0,   9],
       [  0,  21,   0,  11,   0,   0,   0,   0,   0,   7],
       [  0,   3,   0,   4,   0,   0,   0,   0,   0,   1],
       [  0,   8,   0,   6,   0,   0,   0,   0,   0,   9],
       [  0,   4,   0,   2,   0,   0,   0,   0,   0,   4],
       [  0,   4,   0,   3,   0,   0,   0,   0,   0,   2],
       [  0,   1,   0,   0,   0,   0,   0,   0,   0,   1],
       [  0,   5,   0,   2,   0,   0,   0,   0,   0,   4],
       [  0,   5,   0,   2,   0,   0,   0,   0,   0,   2],
       [  0,   6,   0,   2,   0,   0,   0,   0,   0, 127]])